In [1]:
# import necessary libraries
import speech_recognition as sr
from gtts import gTTS
import playsound
import nltk
from nltk.tokenize import word_tokenize
import csv
import os

# Download necessary NLTK data
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lovel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# Load inventory from CSV
def load_inventory(file_path):
    # Initialize an empty dictionary to hold data
    inventory = {}
    # Open CSV file in read mode (r)
    with open(file_path, mode='r') as file:
        reader = csv.DictReader(file)
        # Iterate of each row in the CSV file
        for row in reader:
            # Extract the product name from the current row
            product = row["product"]
            # Add the product to the inventory dictionary with its qty and price
            inventory[product] = {"quantity": int(row["quantity"]), "price": float(row["price"])}
            # Return populated inventory dictionary
    return inventory

# Update the file path to the resources folder
csv_file_path = os.path.join('Resources', 'mock_inventory.csv')
inventory = load_inventory(csv_file_path)


In [3]:
# create a function that will recognize when person is speaking
def listen():
    # Creating a Recognizer instance to recognize speech
    recognizer = sr.Recognizer()
    # Use computer microphone as the source of the audio
    with sr.Microphone() as source:
        # this lets the user know that the assistant is listening
        print("Listening...")
        # captures the audio that comes in from mic
        audio = recognizer.listen(source)
        try:
            # Uses Google's speech recognition to transcribe the audio to text
            text = recognizer.recognize_google(audio)
            # prints the transcribed text
            print(f"User said: {text}")
            # return the transcribed text
            return text
        except sr.UnknownValueError:
            # how to handle where speech was not understood
            print("Sorry, I did not understand that.")
            # empty string shows a failure to understand
            return ""
        except sr.RequestError:
            print("Sorry, the service is down.")
            return ""

def speak(text):
    # Creates a gTTS (Google Text-to-Speech) instance with provided text
    tts = gTTS(text=text, lang='en')
    # Define the filename for the audio file
    filename = "response.mp3"
    # Save to mp3 file
    tts.save(filename)
    # play the mp3 file to speak the captured text
    playsound.playsound(filename)


In [4]:
# create a function that will understand commands and reply with appropriate response
# this will need to be updated as of 7/21, some responses are not relevant.
def process_command(command):
    # tokenize the command into single words and convert them to lowercase
    tokens = word_tokenize(command.lower())
    
    if "find" in tokens and "product" in tokens:
        # If the command has both "find" and "product", respond with:
        speak("Sure, I can help you find a product. What are you looking for?")
    elif "check" in tokens and "stock" in tokens:
        # If the command has both "check and "stock, attempt to find the product
        product_name = extract_product_name(tokens)
        if product_name in inventory:
            # If the product is found in the inventory, respond with the quantity in stock
            quantity = inventory[product_name]["quantity"]
            speak(f"We have {quantity} {product_name}(s) in stock.")
        else:
            # If the product is not found in the inventory, respond with:
            speak(f"Sorry, we don't have {product_name} in our inventory.")
    elif "information" in tokens and "paint" in tokens:
        # If the command contains both :information" and "paint", ask what specific information is needed
        speak("I can provide information about our paints and finishes. What would you like to know?")
    elif "paint" in tokens and "colors" in tokens:
        # this will be removed
        speak("We have a wide variety of paint colors. Are you looking for interior or exterior paints?")
    else:
        # If the command does not match anything...respond with:
        speak("I'm sorry, I didn't understand that. Can you please repeat?")

def extract_product_name(tokens):
    # Iterate over each token in the command
    for token in tokens:
        # Check if the token matches any product name in inventory
        for product in inventory.keys():
            if token in product.lower():
                # Return the product name if a match is found
                return product
            # Return an empty string if no match is found
    return ""


In [5]:
# This block of code ensures that the following statements are only executed 
# when this script is run directly, and not when it is imported as a module in another script.
if __name__ == "__main__":
    # This creates an infinite loop to continuously listen for and process commands.
    while True:
        # Call the listen() function to capture and transcribe spoken commands from the user.
        # The transcribed text is stored in the variable 'command'.
        command = listen()
        # Check if a command was successfully transcribed (i.e., if 'command' is not empty).
        if command:
            # If a valid command was captured, call the process_command() function to handle
            # the command and provide the appropriate response
            process_command(command)


Listening...
User said: find a product



    Error 263 for command:
        open response.mp3
    The specified device is not open or is not recognized by MCI.

    Error 263 for command:
        close response.mp3
    The specified device is not open or is not recognized by MCI.
Failed to close the file: response.mp3


PlaysoundException: 
    Error 263 for command:
        open response.mp3
    The specified device is not open or is not recognized by MCI.